In [114]:
from collections import defaultdict, Counter

In [227]:
def read_file(filename):
    
    food_ingredient = defaultdict(set)
    allergen_food = defaultdict(set)
    
    all_ingredients = []
    all_allergens = []
    unique_ingredients = set()
    unique_allergens = set()
    
    with open(filename, 'r') as f:
        
        food_idx = 0
        line = f.readline()
        
        while line:
            
            line_allergens = line.split('(')[1][8:-2].replace(' ', '').split(',')
            unique_allergens.update(line_allergens)
            all_allergens += line_allergens
            
            line_ingredients = line.split('(')[0].split(' ')[:-1]
            unique_ingredients.update(line_ingredients)
            all_ingredients += line_ingredients
            
            for line_allergen in line_allergens:
                allergen_food[line_allergen].add(food_idx)
                
            food_ingredient[food_idx] = set(line_ingredients)
            
            # Done, next line.
            line = f.readline()
            food_idx += 1
        
        return allergen_food, food_ingredient, unique_ingredients, unique_allergens, all_ingredients

In [228]:
def some_func(allergen_food, food_ingredient, unique_ingredients, unique_allergens):
    allergen_ingredient = defaultdict(set)
    for ua in unique_allergens:
        allergen_ingredient[ua] = set(unique_ingredients)

    for allergen in allergen_food:
        foods = allergen_food[allergen]
        for food in foods:
            ingredients = food_ingredient[food]
            allergen_ingredient[allergen] = set.intersection(allergen_ingredient[allergen],
                                                             ingredients)

    repeat = True
    while repeat:
        #print(allergen_ingredient)  
        repeat = False
        for allergen, ingredients in allergen_ingredient.items():
            if len(ingredients) == 1:
                ingredient = tuple(ingredients)[0]
                for other_allergen, other_ingredients in allergen_ingredient.items():
                    if (ingredient in other_ingredients) and len(other_ingredients) > 1:
                        other_ingredients.remove(ingredient)
                        repeat = True

    #print(allergen_ingredient)
    return allergen_ingredient

In [229]:
def read_some_func(filename):
    allergen_food, food_ingredient, unique_ingredients, unique_allergens, all_ingredients = read_file(filename)
    allergen_ingredient = some_func(allergen_food, food_ingredient, unique_ingredients, unique_allergens)
    
    allergenic_ingredients = set()
    for ingredient in allergen_ingredient.values():
        allergenic_ingredients.update(ingredient)    

    free_ingredients = set.difference(set(unique_ingredients), allergenic_ingredients)
    
    counts = Counter(all_ingredients)
    total = 0
    for free in free_ingredients:
        total += counts[free]
    
    return free_ingredients, total, unique_ingredients, allergenic_ingredients, allergen_food, unique_allergens, food_ingredient

In [230]:
read_some_func('small-input')

({'kfcds', 'nhms', 'sbzzf', 'trh'},
 5,
 {'fvjkl', 'kfcds', 'mxmxvkd', 'nhms', 'sbzzf', 'sqjhc', 'trh'},
 {'fvjkl', 'mxmxvkd', 'sqjhc'},
 defaultdict(set, {'dairy': {0, 1}, 'fish': {0, 3}, 'soy': {2}}),
 {'dairy', 'fish', 'soy'},
 defaultdict(set,
             {0: {'kfcds', 'mxmxvkd', 'nhms', 'sqjhc'},
              1: {'fvjkl', 'mxmxvkd', 'sbzzf', 'trh'},
              2: {'fvjkl', 'sqjhc'},
              3: {'mxmxvkd', 'sbzzf', 'sqjhc'}}))

In [243]:
free_ingredients, total, unique_ingredients, allergenic_ingredients, allergen_food, unique_allergens, food_ingredient = read_some_func('input.txt')

In [232]:
print(total)

2410


In [233]:
print(len(free_ingredients))

192


In [234]:
print(len(unique_ingredients))

200


In [235]:
print(allergenic_ingredients)

{'flnhl', 'pdpgm', 'zrvtg', 'tmp', 'cdslv', 'ttkn', 'mkpmkx', 'vxzpfp'}


## Part 2

In [236]:
import itertools
from copy import deepcopy

In [249]:
free_ingredients, total, unique_ingredients, allergenic_ingredients, allergen_food, unique_allergens, food_ingredient = read_some_func('input.txt')

requires = defaultdict(set)
for allergen, foods in allergen_food.items():
    for food in foods:
        requires[food].add(allergen)

allergen_permutations = itertools.permutations(allergenic_ingredients)
for permutation in allergen_permutations:
    # assign allergens via default order
    ingredient_allergen = {}
    for i, allergen in enumerate(unique_allergens):
        ingredient_allergen[permutation[i]] = allergen
        
    allergen_ingredient = {v: k for k, v in ingredient_allergen.items()}
    
    #print(ingredient_allergen)
    
    # Remove free
    tmp_food_ingredient = deepcopy(food_ingredient)
    for food in tmp_food_ingredient:
        toremove = set()
        for ingredient in tmp_food_ingredient[food]:
            if ingredient in free_ingredients:
                #food_ingredient[food].remove(ingredient)
                toremove.add(ingredient)
        tmp_food_ingredient[food].difference_update(toremove)
                
    # Assign permutation
    tmp_requires = deepcopy(requires)
    #print(requires)
    for food, ingredients in tmp_food_ingredient.items():
        for ingredient in ingredients:
            allergen = ingredient_allergen[ingredient]
            if allergen in tmp_requires[food]:
                tmp_requires[food].remove(allergen)
            
    
    # Check requires
    all_required = True
    for food, allergens in tmp_requires.items():
        if len(allergens) > 0:
            all_required = False
            break
            
    if all_required:
        solution = ingredient_allergen
        print(",".join(sorted(solution, key=lambda k: solution[k])))
        print(tmp_requires)
        break

tmp,pdpgm,cdslv,zrvtg,ttkn,mkpmkx,vxzpfp,flnhl
defaultdict(<class 'set'>, {0: {'sesame', 'shellfish'}, 33: {'sesame', 'shellfish'}, 2: {'sesame', 'shellfish', 'fish'}, 34: {'shellfish'}, 39: {'shellfish', 'fish'}, 11: {'shellfish'}, 14: {'shellfish', 'peanuts', 'nuts'}, 15: {'shellfish'}, 16: {'shellfish', 'fish'}, 19: {'shellfish', 'dairy'}, 20: {'sesame', 'shellfish', 'soy'}, 23: {'sesame', 'shellfish'}, 25: {'shellfish', 'fish', 'dairy'}, 28: {'wheat', 'shellfish'}, 6: {'sesame'}, 8: {'sesame', 'soy'}, 40: {'sesame', 'nuts', 'soy'}, 18: {'sesame', 'soy', 'dairy'}, 31: {'sesame', 'nuts', 'dairy'}, 1: {'fish'}, 37: {'nuts', 'fish', 'soy'}, 26: {'wheat', 'nuts', 'fish'}, 27: {'wheat', 'nuts', 'fish'}, 3: {'nuts'}, 35: {'nuts'}, 41: {'wheat', 'nuts'}, 17: {'nuts'}, 22: {'peanuts', 'nuts'}, 4: {'soy', 'dairy'}, 36: {'wheat', 'dairy'}, 7: {'dairy'}, 21: {'dairy'}, 29: {'peanuts', 'dairy'}, 5: {'wheat', 'soy'}, 13: {'peanuts', 'soy'}, 24: {'soy'}, 30: {'soy'}, 9: {'wheat', 'peanuts'}, 12: 